# Demo Notebook 

This notebook serves the purpose of demonstrating how the Retriaval Augmented Generation improves the LLM code generation. 

## LLM code creation without RAG and CoALA

Firstly, we import the necessary libraries and create an agent object.

In [ ]:
# imports and agent creation
import pandas as pd
from llms.agents.react import ReActAgent
from llms.clients.gpt import GPTClient
from llms.settings import settings

client = GPTClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
    deployment_id="gpt-4-32k",
    max_response_tokens=1000,
    temperature=0.0,
)
agent = ReActAgent(client)

Then we define a prompt and ask the agent without RAG and CoALA implemented for the code. Then we execute the function code and save the function that the agent generated under `agent_func`.

In [ ]:
prompt = """How can I convert this one-hot-encoded dataframe: df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})
into a categorical dataframe?"""

generated_code = agent.run(prompt)

namespace_agent = {}
exec(generated_code, namespace_agent)
agent_func = namespace_agent['response_function']

Next, we define our input and the expected function.

In [ ]:
input = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})

def expected_func(input):
    result = pd.from_dummies(input, sep="_")
    return result

Now we can compare the results of the two functions:

In [ ]:
print('Desired result:')
print(expected_func(input))
print()
print('Agent result:')
print(agent_func(input))

## LLM code creation with RAG and CoALA

We start by creating a new LLM agent that is augmented by RAG and CoALA.

In [ ]:
# import the FAISS vector storage for the retrieval augmented generation and CoALA
from llms.rag.faiss import FAISS
from llms.rag.coala import CoALA

rag = FAISS.create_index_from_texts(
    texts=["print('Hello Wourld')", "None", "Irrelevant content"],
    llm_client=client,
)

code_storage = FAISS.create_index_from_texts(
    texts=["Question: Print Hello World\nFinal Answer:def response_function():\n    print('Hello World')"],
    llm_client=client,
)

coala_rag = CoALA(docs_vector_store=rag, code_vector_store=code_storage)

In [ ]:
# insert client initialisation with RAG and CoALA here and call it !!rag_agent!!
rag_agent = ReActAgent(client, rag=coala_rag)

In [ ]:
# reminder
prompt = """How can I convert this one-hot-encoded dataframe: df = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})
into a categorical dataframe?"""

generated_code_rag = rag_agent.run(prompt)

namespace_rag_agent = {}
exec(generated_code_rag, namespace_rag_agent)
rag_agent_func = namespace_rag_agent['response_function']

As the `input` and `expected_func` stay the same, we can now compare the results of the rag enhanced agent with the expected result.

In [ ]:
input = pd.DataFrame({"col1_a": [1, 0, 1], "col1_b": [0, 1, 0], "col2_a": [0, 1, 0], "col2_b": [1, 0, 0], "col2_c": [0, 0, 1]})

In [ ]:
print('Desired result:')
print(expected_func(input))
print()
print('Rag enhanced agent result:')
print(rag_agent_func(input))